In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast'

In [2]:
from typing import Union, Optional
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.datasets import fetch_dataset
import numpy as np
import pandas as pd
from astral.sun import sun
from astral import LocationInfo
from skforecast.datasets import fetch_dataset


In [3]:
def select_features(
    selector: object,
    forecaster: object,
    y: Union[pd.Series, pd.DataFrame],
    exog: Optional[Union[pd.Series, pd.DataFrame]] = None,
    force_lags_selection: bool = False,
    subsample: Union[int, float] = 0.5,
    verbose: bool = True,
) -> Union[list, list]:
    """
    Feature selection using any of the classes from sklearn.feature_selection 
    module (such as RFECV, SelectFromModel, etc).

    Parameters
    ----------
    selector : object
        A feature selector from sklearn.feature_selection.
    forecaster : object
        A forecaster from skforecast.
    y : pd.Series or pd.DataFrame   
        Target time series to which feature selection will be applied.
    exog : pd.Series or pd.DataFrame, optional (default=None)   
        Exogenous variables.
    force_lags_selection : bool, optional (default=True)
        Whether to include all lagged variables as part of the selected features.
        If `False` (default), lagged variables are also evaluated by the selector
        and only those that are selected are included in the final list of selected
        features. If `True`, exogenous variables are evaluated without the presence
        of lagged variables and then all lagged variables are added to the selected
        exogenous variables.
    subsample : int or float, optional (default=0.5)
        Number of records to use for feature selection. If int, number of records
        to use. If float, proportion of records to use.
    verbose : bool, optional (default=True)
        Print information about feature selection process.

    Returns
    -------
    selected_lags : list
        List of selected lags.
    selected_exog : list
        List of selected exogenous variables.
    """

    X_train, y_train = forecaster.create_train_X_y(
                            y    = y,
                            exog = exog,
                        )

    if force_lags_selection:
        lags_cols = [col for col in X_train.columns if col.startswith("lag_")]
        X_train = X_train.drop(columns=lags_cols)

    if isinstance(subsample, float):
        subsample = int(len(X_train)*subsample)

    rng = np.random.default_rng(seed=785412)
    sample = rng.choice(X_train.index, size=subsample, replace=False)
    X_train_sample = X_train.loc[sample, :]
    y_train_sample = y_train.loc[sample]
    
    selector.fit(X_train_sample, y_train_sample)
    selected_features = selector.get_feature_names_out()

    if force_lags_selection:
        selected_lags = lags_cols
    else:
        selected_lags = [
            int(feature.replace("lag_", ""))
            for feature in selected_features
            if feature.startswith("lag_")
        ]
        
    selected_exog = [
        feature
        for feature in selected_features
        if not feature.startswith("lag_")
    ]

    if verbose:
        print("Recursive feature elimination")
        print("-----------------------------")
        print(f"Total number of features available: {X_train.shape[1]}") 
        print(f"Total number of records available: {X_train.shape[0]}")
        print(f"Total number of records used for feature selection: {X_train_sample.shape[0]}")
        print(f"Number of features selected: {len(selected_features)}")
        print(f"Selected lags: {selected_lags}")
        print(f"Selected exog : \n {selected_exog}")

    return selected_lags, selected_exog    

In [4]:
# Downloading data
# ==============================================================================
data = fetch_dataset('bike_sharing', raw=True)

# Preprocessing data (setting index and frequency)
# ==============================================================================
data = data[['date_time', 'users', 'holiday', 'weather', 'temp', 'atemp', 'hum', 'windspeed']]
data['date_time'] = pd.to_datetime(data['date_time'], format='%Y-%m-%d %H:%M:%S')
data = data.set_index('date_time')
data = data.asfreq('H')
data = data.sort_index()
data.head()

# Calendar features
# ==============================================================================
calendar_features = pd.DataFrame(index=data.index)
calendar_features['month'] = calendar_features.index.month
calendar_features['week_of_year'] = calendar_features.index.isocalendar().week
calendar_features['week_day'] = calendar_features.index.day_of_week + 1
calendar_features['hour_day'] = calendar_features.index.hour + 1

# Sunlight features
# ==============================================================================
location = LocationInfo(
    name='Washington DC',
    region='USA',
    timezone='US/Eastern',
    latitude=40.516666666666666,
    longitude=-77.03333333333333
)
sunrise_hour = [
    sun(location.observer, date=date, tzinfo=location.timezone)['sunrise'].hour
    for date in data.index
]
sunset_hour = [
    sun(location.observer, date=date, tzinfo=location.timezone)['sunset'].hour
    for date in data.index
]
sun_light_features = pd.DataFrame({
                         'sunrise_hour': sunrise_hour,
                         'sunset_hour': sunset_hour}, 
                         index = data.index
                     )
sun_light_features['daylight_hours'] = (
    sun_light_features['sunset_hour'] - sun_light_features['sunrise_hour']
)
sun_light_features['is_daylight'] = np.where(
                                        (data.index.hour >= sun_light_features['sunrise_hour']) & \
                                        (data.index.hour < sun_light_features['sunset_hour']),
                                        1,
                                        0
                                    )

# Holiday features
# ==============================================================================
holiday_features = data[['holiday']].astype(int)
holiday_features['holiday_previous_day'] = holiday_features['holiday'].shift(24)
holiday_features['holiday_next_day'] = holiday_features['holiday'].shift(-24)

# Temperature features
# ==============================================================================
temp_features = data[['temp']].copy()
temp_features['temp_roll_mean_1_day'] = temp_features['temp'].rolling(24, closed='left').mean()
temp_features['temp_roll_mean_7_day'] = temp_features['temp'].rolling(24*7, closed='left').mean()
temp_features['temp_roll_max_1_day'] = temp_features['temp'].rolling(24, closed='left').max()
temp_features['temp_roll_min_1_day'] = temp_features['temp'].rolling(24, closed='left').min()
temp_features['temp_roll_max_7_day'] = temp_features['temp'].rolling(24*7, closed='left').max()
temp_features['temp_roll_min_7_day'] = temp_features['temp'].rolling(24*7, closed='left').min()


# Merge all exogenous variables
# ==============================================================================
df_exogenous_features = pd.concat([
                            calendar_features,
                            sun_light_features,
                            temp_features,
                            holiday_features
                        ], axis=1)

df_exogenous_features.head(4)

# Cliclical encoding of calendar and sunlight features
# ==============================================================================
def cyclical_encoding(data: pd.Series, cycle_length: int) -> pd.DataFrame:
    """
    Encode a cyclical feature with two new features sine and cosine.
    The minimum value of the feature is assumed to be 0. The maximum value
    of the feature is passed as an argument.
      
    Parameters
    ----------
    data : pd.Series
        Series with the feature to encode.
    cycle_length : int
        The length of the cycle. For example, 12 for months, 24 for hours, etc.
        This value is used to calculate the angle of the sin and cos.

    Returns
    -------
    result : pd.DataFrame
        Dataframe with the two new features sin and cos.

    """

    sin = np.sin(2 * np.pi * data/cycle_length)
    cos = np.cos(2 * np.pi * data/cycle_length)
    result =  pd.DataFrame({
                  f"{data.name}_sin": sin,
                  f"{data.name}_cos": cos
              })

    return result


month_encoded = cyclical_encoding(df_exogenous_features['month'], cycle_length=12)
week_of_year_encoded = cyclical_encoding(df_exogenous_features['week_of_year'], cycle_length=52)
week_day_encoded = cyclical_encoding(df_exogenous_features['week_day'], cycle_length=7)
hour_day_encoded = cyclical_encoding(df_exogenous_features['hour_day'], cycle_length=24)
sunrise_hour_encoded = cyclical_encoding(df_exogenous_features['sunrise_hour'], cycle_length=24)
sunset_hour_encoded = cyclical_encoding(df_exogenous_features['sunset_hour'], cycle_length=24)

cyclical_features = pd.concat([
                        month_encoded,
                        week_of_year_encoded,
                        week_day_encoded,
                        hour_day_encoded,
                        sunrise_hour_encoded,
                        sunset_hour_encoded
                    ], axis=1)

df_exogenous_features = pd.concat([df_exogenous_features, cyclical_features], axis=1)
df_exogenous_features.head(3)

# Interaction between exogenous variables
# ==============================================================================
transformer_poly = PolynomialFeatures(
                       degree           = 2,
                       interaction_only = True,
                       include_bias     = False
                   ).set_output(transform="pandas")

poly_cols = [
    'month_sin', 
    'month_cos',
    'week_of_year_sin',
    'week_of_year_cos',
    'week_day_sin',
    'week_day_cos',
    'hour_day_sin',
    'hour_day_cos',
    'sunrise_hour_sin',
    'sunrise_hour_cos',
    'sunset_hour_sin',
    'sunset_hour_cos',
    'daylight_hours',
    'is_daylight',
    'holiday_previous_day',
    'holiday_next_day',
    'temp_roll_mean_1_day',
    'temp_roll_mean_7_day',
    'temp_roll_max_1_day',
    'temp_roll_min_1_day',
    'temp_roll_max_7_day',
    'temp_roll_min_7_day',
    'temp',
    'holiday'
]

poly_features = transformer_poly.fit_transform(df_exogenous_features[poly_cols].dropna())
poly_features = poly_features.drop(columns=poly_cols)
poly_features.columns = [f"poly_{col}" for col in poly_features.columns]
poly_features.columns = poly_features.columns.str.replace(" ", "__")
df_exogenous_features = pd.concat([df_exogenous_features, poly_features], axis=1)
df_exogenous_features.head(4)

# Select exogenous variables to be included in the model
# ==============================================================================
exog_features = []
# Columns that ends with _sin or _cos are selected
exog_features.extend(df_exogenous_features.filter(regex='_sin$|_cos$').columns.tolist())
# columns that start with temp_ are selected
exog_features.extend(df_exogenous_features.filter(regex='^temp_.*').columns.tolist())
# Columns that start with holiday_ are selected
exog_features.extend(df_exogenous_features.filter(regex='^holiday_.*').columns.tolist())
exog_features.extend(['temp', 'holiday'])

df_exogenous_features = df_exogenous_features.filter(exog_features, axis=1)

bike_sharing
------------
Hourly usage of the bike share system in the city of Washington D.C. during the
years 2011 and 2012. In addition to the number of users per hour, information
about weather conditions and holidays is available.
Fanaee-T,Hadi. (2013). Bike Sharing Dataset. UCI Machine Learning Repository.
https://doi.org/10.24432/C5W894.
Shape of the dataset: (17544, 12)


In [5]:
df_exogenous_features = df_exogenous_features.dropna()
data = data.loc[df_exogenous_features.index, :]

In [6]:
forecaster = ForecasterAutoreg(
                regressor = Ridge(random_state=123),
                lags      = 169,
            )

selector = RFECV(
    estimator              = forecaster.regressor,
    min_features_to_select = 1,
    cv                     = 5,
)

selected_lags, selected_exog = select_features(
    selector             = selector,
    forecaster           = forecaster,
    y                    = data['users'],
    exog                 = df_exogenous_features,
    force_lags_selection = False,
    subsample            = 0.1,
    verbose              = True,
)        

Recursive feature elimination
-----------------------------
Total number of features available: 257
Total number of records available: 17183
Total number of records used for feature selection: 1718
Number of features selected: 106
Selected lags: [1, 2, 3, 8, 9, 22, 24, 25, 47, 96, 97, 120, 121, 144, 145, 146, 147, 159, 160, 161, 162, 166, 167, 168, 169]
Selected exog : 
 ['month_sin', 'month_cos', 'week_of_year_sin', 'week_of_year_cos', 'week_day_sin', 'week_day_cos', 'hour_day_sin', 'hour_day_cos', 'sunrise_hour_sin', 'sunrise_hour_cos', 'sunset_hour_sin', 'sunset_hour_cos', 'poly_month_sin__month_cos', 'poly_month_sin__week_of_year_sin', 'poly_month_sin__week_of_year_cos', 'poly_month_sin__week_day_sin', 'poly_month_sin__week_day_cos', 'poly_month_sin__hour_day_sin', 'poly_month_sin__hour_day_cos', 'poly_month_sin__sunrise_hour_sin', 'poly_month_sin__sunrise_hour_cos', 'poly_month_sin__sunset_hour_sin', 'poly_month_sin__sunset_hour_cos', 'poly_month_cos__week_day_sin', 'poly_month_co

In [7]:
from sklearn.feature_selection import SelectFromModel

selector = SelectFromModel(
    estimator    = forecaster.regressor,
    threshold    = 0.25,
    max_features = 25
)

selected_lags, selected_exog = select_features(
    selector             = selector,
    forecaster           = forecaster,
    y                    = data['users'],
    exog                 = df_exogenous_features,
    force_lags_selection = False,
    subsample            = 0.1,
    verbose              = True,
)  

Recursive feature elimination
-----------------------------
Total number of features available: 257
Total number of records available: 17183
Total number of records used for feature selection: 1718
Number of features selected: 25
Selected lags: []
Selected exog : 
 ['week_of_year_sin', 'week_day_sin', 'poly_month_sin__month_cos', 'poly_month_sin__week_of_year_sin', 'poly_month_sin__week_of_year_cos', 'poly_month_sin__hour_day_cos', 'poly_month_sin__sunrise_hour_sin', 'poly_month_sin__sunrise_hour_cos', 'poly_month_cos__week_day_cos', 'poly_month_cos__hour_day_cos', 'poly_week_of_year_sin__hour_day_cos', 'poly_week_of_year_sin__sunrise_hour_cos', 'poly_week_of_year_cos__week_day_sin', 'poly_week_of_year_cos__week_day_cos', 'poly_week_of_year_cos__hour_day_sin', 'poly_week_of_year_cos__hour_day_cos', 'poly_week_of_year_cos__sunrise_hour_cos', 'poly_week_day_sin__sunrise_hour_sin', 'poly_week_day_cos__sunrise_hour_cos', 'poly_week_day_cos__sunset_hour_sin', 'poly_week_day_cos__sunset_hour

In [8]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import ShuffleSplit

selector = SequentialFeatureSelector(
    estimator    = forecaster.regressor,
    n_features_to_select = 25,
    direction    = 'forward',
    cv           = ShuffleSplit(n_splits=1, test_size=0.3, random_state=951),
    scoring      = 'neg_mean_absolute_error',
    n_jobs       = -1
)

selected_lags, selected_exog = select_features(
    selector             = selector,
    forecaster           = forecaster,
    y                    = data['users'],
    exog                 = df_exogenous_features,
    force_lags_selection = False,
    subsample            = 0.1,
    verbose              = True,
)  

KeyboardInterrupt: 

In [ ]:
# def feature_selection_rfecv(
#     forecaster: object, 
#     y: Union[pd.Series, pd.DataFrame],
#     exog: Optional[Union[pd.Series, pd.DataFrame]] = None,
#     min_features_to_select: int = 1,
#     cv: Union[int, object, None] = None,
#     select_lags: bool = True,
#     subsample: Union[int, float] = 0.5,
#     verbose: bool = True,
#     **kwargs
# ) -> list:
#     """
#     Recursive feature elimination with cross-validation to select features based
#     on sklearn.feature_selection.RFECV.

#     Given a regressor that assigns weights to features (such as coef_, 
#     feature_importances_), the goal of recursive feature elimination (RFE) is to
#     select features by recursively considering smaller and smaller sets of features.
#     First, the regressor is trained with all the available features and the importance
#     of each feature is obtained. Then, the least important features are pruned
#     from the current set of features and the performance of the regressor is
#     evaluated using cross-validation, if the performance of the model has not
#     decreased, then the pruned features are discarded. The process is repeated
#     until the desired number of features to select is reached or there are no
#     more features to discard. 
    
#     Parameters
#     ----------
#     forecaster : object
#         A forecaster from skforecast.
#     y : pd.Series or pd.DataFrame
#         Target time series to which feature selection will be applied.
#     exog : pd.Series or pd.DataFrame, optional (default=None)
#         Exogenous variables.
#     min_features_to_select : int, optional (default=1)
#         The minimum number of features to be selected.
#     include_lags: bool, optional (default=True)
#         Whether to include lagged variables in feature selection. If `False`, exogenous
#         variables are evaluated without the presence of lagged variables.
#     subsample : int or float, optional (default=0.5)
#         Number of records to use for feature selection. If int, number of records
#         to use. If float, proportion of records to use.
#     cv : int, cross-validation generator or an iterable, default=None
#         Determines the cross-validation splitting strategy. Possible inputs for cv are:
#         - None, to use the default 5-fold cross-validation
#         - integer, to specify the number of folds
#         - CV splitter
#         - An iterable yielding (train, test) splits as arrays of indices    
#     **kwargs : optional
#         Aditional arguments of sklearn.feature_selection.RFECV.

#     Returns
#     -------
#     selected_features : list
#         List of selected features.
#     """

#     X_train, y_train = forecaster.create_train_X_y(
#                         y    = y,
#                         exog = exog,
#                     )

#     if not select_lags:
#         lags_cols = [col for col in X_train.columns if col.startswith("lag_")]
#         X_train = X_train.drop(columns=lags_cols)

#     if isinstance(subsample, float):
#         subsample = int(len(X_train)*subsample)

#     rng = np.random.default_rng(seed=785412)
#     sample = rng.choice(X_train.index, size=subsample, replace=False)
#     X_train_sample = X_train.loc[sample, :]
#     y_train_sample = y_train.loc[sample]

#     selector = RFECV(
#         estimator              = forecaster.regressor,
#         min_features_to_select = min_features_to_select,
#         cv                     = cv,
#         **kwargs
#     )
#     selector.fit(X_train_sample, y_train_sample)
#     selected_features = selector.get_feature_names_out()
#     selected_lags = [
#         int(feature.replace("lag_", ""))
#         for feature in selected_features
#         if feature.startswith("lag_")
#     ]
#     selected_exog = [
#         feature
#         for feature in selected_features
#         if not feature.startswith("lag_")
#     ]

#     if verbose:
#         print("Recursive feature elimination")
#         print("-----------------------------")
#         print(f"Total number of features available: {X_train.shape[1]}") 
#         print(f"Total number of records available: {X_train.shape[0]}")
#         print(f"Total number of records used for feature selection: {X_train_sample.shape[0]}")
#         print(f"Number of features selected: {len(selected_features)}")
#         print(f"Selected lags: {selected_lags}")
#         print(f"Selected exog : \n {selected_exog}")

#     return selected_lags, selected_exog
    

In [ ]:
# feature_selection_rfecv(
#     forecaster             = forecaster,
#     y                      = data['users'],
#     exog                   = df_exogenous_features,
#     select_lags            = True,
#     min_features_to_select = 25,
#     cv                     = 3,
#     subsample              = 0.5,
#     step                   = 1,
#     scoring                = 'neg_mean_absolute_error',
#     n_jobs                 = -1
# )